In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

In [ ]:

netgen_mesh = unit_square.GenerateMesh(maxh=1)
#netgen_mesh.Refine()
mesh=Mesh(netgen_mesh)
# mesh.Refine(onlyonce=False)
mesh.Refine(onlyonce=False)
Draw(mesh)
elvol = Integrate(CoefficientFunction(1),mesh,element_wise=True)
h = [(2*vol)**(1/2) for vol in elvol]
print(min(h),max(h))


Set correct functionspaces in NGSolve, which will give the corresponding FE Elements, for l=0 its just H1 wihch will give the normal lagrangian elements, which for lowest order are the whitney 0 forms.

In [ ]:
#set function space, for l=0 its normal lagrangian finite elements
H_h = Compress(HDiv(mesh, order = 1, RT=True)) # main function space
H_H = Compress(HDiv(mesh, order = 2, RT=True)) # high order Function space for Riesz representative


In [ ]:
#choose w_h and set its values on the nodes
w = (sin(x), cos(y))

w_h = GridFunction(H_h) #set the nodes
w_h.Set(w)

f_H = GridFunction(H_H, name="f_H") #high order 
#w_H.Set(w) #dont need to set the high order gridfunction

#Draw(w_H)

Create the galerkin matrices used to calculate the Riesz representative

In [ ]:
#low order
u_h = H_h.TrialFunction() # define the trial function
v_h = H_h.TestFunction() # define the test function

#High order
u_H = H_H.TrialFunction() # define the trial function
v_H= H_H.TestFunction() # define the test function

m = BilinearForm(H_H) #define m with the L2 inner product on the high order space
m += u_H*v_H * dx 
m.Assemble()

a = BilinearForm(H_H) # define the H1 inner product on the high order space
a += div(u_H) * div(v_H) * dx
a += u_H*v_H * dx 
a.Assemble()

Approximate the dual norm, by the Riesz representive.
Get the Riesz Representive by solving a LGS, since we're approximating we set the space of the Riesz represntative in a higher order space (higher polynomial degree)

In [ ]:
#create embedding matrix T

# a_mixed = BilinearForm(trialspace=H_h, testspace=H_H)

# a_dest = BilinearForm(H_H)

# v_H_dual = v_H.Operator("dual")

# dS = dx(element_boundary=True)
# a_dest += u_H*v_H_dual*dx + u_H*v_H_dual*dS
# a_dest.Assemble()

# a_mixed += u_h*v_H_dual*dx + u_h*v_H_dual*dS
# a_mixed.Assemble()

# #build T
# t = a_dest.mat.Inverse() @ a_mixed.mat

############################# doing it directly as a L2 projection

a_mixed = BilinearForm(trialspace=H_h, testspace=H_H)
a_mixed += u_h*v_H*dx
a_mixed.Assemble()
print(a_mixed.mat.shape)

a_dest = BilinearForm(H_H)
a_dest += u_H*v_H*dx
a_dest.Assemble()

t = a_dest.mat.Inverse() @ a_mixed.mat

#w_H is the high order embedding of w_h
w_H = GridFunction(H_H, name="w_H")
w_H.vec.data = t * w_h.vec

Draw(w_h)
Draw(w_H)


######################## this works too

#this feels weird, find way to get the range from space H_h itself
# mixed = FESpace([H_h])

# emb = Embedding(H_H.ndof, mixed.Range(0))

# w_H.vec.data = emb * w_h.vec 

# Draw(w_H)

#results in same shape
#print(t.shape)
#print(emb.shape)



In [ ]:
a_inv = a.mat.Inverse(freedofs=H_H.FreeDofs())

#does the same as below
#f_H.vec.data = a_inv @ m.mat * w_H.vec
#Draw(f_H)

#solve for the riesz representative (f_H) 
temp = f_H.vec.CreateVector()
temp.data = m.mat * w_H.vec
f_H.vec.data = a_inv * temp
Draw(f_H)



then plug the Riesz representive in the GEV problem and solve it, where the largest Eigenvalue will be the constant we're looking for

In [ ]:
#create Matrices for GEV problem

#c is the low order galerkin matrix
c = BilinearForm(H_h)
c += div(u_h) * div(v_h) * dx
c += u_h*v_h * dx 
c.Assemble()

#check norms
temp3 = c.mat * w_h.vec
w_h_norm = w_h.vec.InnerProduct(temp3)
print("H(div) norm of w_h:", sqrt(w_h_norm)) #norm of w_h in H(div,

temp4 = a.mat * f_H.vec
f_H_norm = f_H.vec.InnerProduct(temp4)
print("H(div) norm of f_H, the riesz representative of w_h:", sqrt(f_H_norm)) #norm of the riesz representative

#b = t.T @ m.mat.T @ a_inv @ m.mat @ t

C = c.mat.ToDense().NumPy()
M = m.mat.ToDense().NumPy()
A_inv = a_inv.ToDense().NumPy()
T = t.ToDense().NumPy()

#M == M.T 
B = T.T @ M.T @ A_inv @ M @ T

In [ ]:
#solve GEV using the scipy solver s


y = sp.linalg.eigvals(B,C)
print("maximal Eigenvalue:",max(y))
print("minimal Eigenvalue:",min(y))
